In [ ]:
from sbi.inference import SNLE, SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.simulators.linear_gaussian import (
    linear_gaussian,
    samples_true_posterior_linear_gaussian_uniform_prior,
)
import torch
from sbi.utils import BoxUniform
import matplotlib.pyplot as plt
from sbi.analysis import pairplot

# What could go wrong (and how to fix it)

## 1. Stay in the SBI bubble: assume valid simulator and prior

<img src="janfb/figures/bubble_zacktionman_Flickr.jpg" align="center" alt="beadexample" width="500"/>

<img src="janfb/figures/bubble_zacktionman_Flickr.jpg" align="right" width="100"/>

### What could go wrong?

- the density estimator is off

- in case of likelihood based approaches: MCMC samples are off


<img src="janfb/figures/bubble_zacktionman_Flickr.jpg" align="right" width="100"/>

### Possible reasons

1) training not converge properly / too little training data
    
2) density estimator lacks flexibility

3) summary statistics (or embedding net) not informative


<img src="janfb/figures/bubble_zacktionman_Flickr.jpg" align="right" width="100"/>

### Diagnostics?

- training logs:
    - check validaton loss for convergence
    - ...

- (MCMC convergence statistics)

- posterior predictive checks
    - sample from posterior and simulate
    - compare to $x_o$
    
- simulation-based calibration

## Problem 1: Training not converged

In [ ]:
# Gaussian simulator
def simulator(theta, scale=0.5):
    # Sample from standard normal, shift with mean.
    return scale * torch.randn(theta.shape) + theta

num_dim = 3
simulator_scale = 0.5
num_samples = 1000
# Uniform prior.
prior = BoxUniform(-5 * torch.ones(num_dim), 5 * torch.ones(num_dim))
x_o = torch.ones(1, num_dim)
# True posterior
true_samples = simulator_scale * torch.randn(num_samples, num_dim) + x_o

In [ ]:
# run simulations
num_simulations = 20  # Little training data.
theta = prior.sample((num_simulations,))
x = simulator(theta, scale=simulator_scale)

In [ ]:
# run inference
inferer = SNPE(prior, density_estimator="mdn").append_simulations(theta, x)
density_estimator = inferer.train()
posterior = inferer.build_posterior()

### SBI posterior is off

In [ ]:
# Draw posterior samples and plot 1 and 2-D marginals.
posterior_samples = posterior.sample((num_samples,), x=x_o)
pairplot([posterior_samples, true_samples], upper="scatter", limits=[[-5, 5]], figsize=(8, 8));
plt.legend(['sbi-posterior' ,'true-posterior']);

## Diagnostics

### SBI training logs with Tensorboard

- In a separate terminal window in the same folder run:

`tensorboard --logdir sbi-logs/`

- This will open a Tensorboard on a localhost, usually http://localhost:6006/

- Demo...

## Diagnostics

### Posterior predictive checks

- General idea: samples from the posterior should reproduce the observed data $x_o$
    - plus simulator noise

- Samples from the posterior plugged into the simulator should cluster around $x_o$


In [ ]:
# Simulate true posterior samples
posterior_predictive_samples = simulator(true_samples)
# Plot on top of x_o
pairplot([posterior_predictive_samples], upper="scatter", points_colors="k", points=x_o, limits=[[-5, 5]]);

## Practical 1 (10 min):

1. Run the inference as it is
2. Start Tensorboard and inspect the training logs
    - especially training log probs and validation log probs
    - what do you observe? did the training converge properly?
3. Run posterior predictive checks
4. Change the training settings and re-run the inference
5. Repeat the checks, did it help?

## Problem 2: Choice of density estimator

- SBI offers different types of density estimators: 
    - mixture density networks (of Gaussians) (MDN)
    - normalizing flows

- MDN are fast in training, sampling and evaluation
- Flows are more flexible

## Normalizing flows

<img src="janfb/figures/nutshell_Kletr-Shutterstock.jpg" align="left" alt="beadexample" width="300"/> 

- transform a simple base distribution to a complex target distribution

- transforms can be trained with NNs (with certain assumptions)

- concatenating transforms -> powerful (conditional) density estimator

## Example 2: Inference on the two-moon task

In [ ]:
# Load two-moon task from sbi-benchmark package
import sbibm
task = sbibm.get_task("two_moons")
simulator = task.get_simulator()
prior = task.get_prior_dist()

In [ ]:
# Simulate
num_simulations = 10000
num_samples = 1000
theta = prior.sample((num_simulations,))
x = simulator(theta)

In [ ]:
# run inference with MDN
inferer = SNPE(prior, density_estimator="mdn").append_simulations(theta, x)
density_estimator = inferer.train()
posterior = inferer.build_posterior(density_estimator)

### MDN posterior fails to learn the two moons

In [ ]:
# Draw posterior samples and plot 1 and 2-D marginals.
x_o = task.get_observation(1)
mdn_samples = posterior.sample((num_samples,), x=x_o)
true_samples = task.get_reference_posterior_samples(1)[:num_samples,]
pairplot([mdn_samples, true_samples], upper="scatter", limits=[[-1, 1]], figsize=(7, 7));
plt.legend(["mdn-posterior", "true-posterior"]);

## Practical 2: Changing density estimators (10 min)

### Tasks
1. Change the density estimator to a flow and train again
2. Compare to the reference posterior samples using `pairplot`.
3. [Optional] Have a look at `sbi.utils.get_nn_models` to see how to build a customised density estimator using the function `posterio_nn(...)`.

# What could go wrong and how to fix it

## 2. Take a step back: look at the full Bayesian workflow
<img src="janfb/figures/what-if-i-told-you-this-is-not-fully-bayesian.jpg" align="center" alt="beadexample" width="500"/>

## References

### Further reading
- 

### Figures
- bubble: 
- nutshell:
- GIF: